In [1]:
from requests_html import HTMLSession
import json
import sqlite3
import sqlalchemy
import time
session=HTMLSession()

In [2]:
#获取配置
with open('caiji.config','r') as f:
    configjson=f.read()
configdict=json.loads(configjson)

with open('time.config','r') as f:
    timejson=f.read()
timedict=json.loads(timejson)

In [2]:
con=sqlite3.connect('spyder.db')
creatvideosql='''CREATE TABLE video (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  author_id INTEGER NOT NULL,
  name TEXT NOT NULL,
  img TEXT,
  jishu TEXT,
  othername TEXT,
  director TEXT,
  actor TEXT,
  genre TEXT,
  region TEXT,
  language TEXT,
  screen TEXT,
  mlen TEXT,
  updatet TEXT,
  clicknum TEXT,
  todayclicknum TEXT,
  score TEXT,
  scorenum TEXT,
  introduction TEXT,
  yun1 TEXT,
  m3u8 TEXT,
  download TEXT,
  platform TEXT,
  other TEXT,
  other1 TEXT,
  other2 TEXT,
  other3 TEXT,
  permituid TEXT,
  FOREIGN KEY (author_id) REFERENCES user (id)
);'''

c=con.cursor()
tablelist=c.execute("select * from sqlite_master where type = 'table' and name = 'video'").fetchall()
if not tablelist:
    c.execute(creatvideosql)
    con.commit()

In [4]:
a=["名称","封面","集数","别名","导演","主演","类型","地区","语言","上映","片长","更新","总播放量","今日播放量","总评分","评分次数","简介","yun1","m3u8","下载","来源"]
e=["name","img","jishu","othername","director","actor","genre","region","language","screen","mlen","updatet","clicknum","todayclicknum","score","scorenum","introduction","yun1","m3u8",            "download","platform"]


In [5]:
updatelist=[]
savepath='./data/'
for keyset,valset in configdict.items():
    pagenum=1
    url_av=[]

    url=keyset
    print(url)
    
    stopflag=1
    righttime=None
    deadline=float(timedict[url]['deadline'])# 提取截至日期
    while(stopflag):
        try :
            r=session.get(url)
        except:
            print('url'+'not response')
            continue
        
        if righttime == None:
            righttime=time.time()
        urllist=r.html.links#获取所有链接
        for temp in urllist:#寻找详情页的url列表
            if configdict[keyset]['useful_url'] in temp:
                url_av.append(temp)

        for suburl in url_av:#电影详情页
            try:
                ph=session.get(keyset+suburl)
            except:
                print(keyset+suburl+'not response')
                continue
            moviedict={}#一条数据信息
            moviedict["platform"]=keyset
            for info,rule in configdict[keyset]['target_xpath'].items():#获取当前电影的详细信息
                if info =="img":
                    if(ph.html.xpath(rule)):
                        moviedict[info]=ph.html.xpath(rule)[0]
                    else:
                        moviedict[info]=''
                elif info =="m3u8" or info=="yun1" or info == "download":
                    moviedict[info]={}
                    for subset in ph.html.xpath(rule):
                        sep=subset.text.find('$')
                        if sep:
                            subsetlabel=subset.text[:sep]
                            subseturl=subset.text[sep+1:]
                            moviedict[info][subsetlabel]=subseturl
                        else:
                            moviedict[info][subsetlabel]=subset
                elif info=="name":
                    tempname=ph.html.xpath(rule)[0].text
                    tempname=tempname.split(' ')
                    moviedict[info]=tempname[0]

                else:
                    if(ph.html.xpath(rule)):
                        moviedict[info]=ph.html.xpath(rule)[0].text
                    else:
                        moviedict[info]=''
            updatelist.append(moviedict)
            updatet=time.mktime(time.strptime(moviedict['updatet'],"%Y-%m-%d %H:%M:%S"))
            if updatet<deadline:
                stopflag=0
                break

        


        #next page
        pagenum+=1
        url=configdict[keyset]+configdict[keyset]['nextpage']%pagenum



        timedict[url]['deadline']=str(righttime)
        json.dump(timedict, open('time.config', "w"))

http://www.okzy.co/


In [21]:
ph=session.get('https://www.rrzyw.cc/index.php/vod/detail/id/16936.html')

for i in ph.html.xpath("//span[@class='text-red hidden-xs']"):
    print(i.text)

2020-04-09 03:04:41


In [24]:
keyset+configdict[keyset]['nextpage']%2

'https://www.rrzyw.cc//index.php/index/index/page/2.html'

In [23]:
configdict[keyset]['nextpage']

'/index.php/index/index/page/%d.html'

In [6]:

def insertdb(c,jsondata):#将json文件插入数据库
    columnname=''
    values=""
    para=[]
    for key,vals in jsondata.items():
        columnname+=(key+',')
        values+='?,'
        if isinstance(vals, dict):
            tempvals=json.dumps(vals)
            para.append(tempvals)
        else:
            para.append(jsondata[key])

    columnname+='author_id'
    values+='?'
    para.append(1)
    tempsql='INSERT INTO video (%s) VALUES (%s)'%(columnname,values)
    print(tempsql)
    c.execute(tempsql,para)


for tempdata in updatelist:
    sqlres=c.execute("SELECT id,platform,screen FROM video WHERE name=?",[tempdata['name']]).fetchall()
    if(not sqlres):
        print('ok')
        insertdb(c,tempdata)



img,jishu,othername,director,actor,genre,region,language,screen,mlen,updatet,clicknum,todayclicknum,score,scorenum,introduction,yun1,m3u8,download,author_id) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
ok
INSERT INTO video (platform,name,img,jishu,othername,director,actor,genre,region,language,screen,mlen,updatet,clicknum,todayclicknum,score,scorenum,introduction,yun1,m3u8,download,author_id) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
ok
INSERT INTO video (platform,name,img,jishu,othername,director,actor,genre,region,language,screen,mlen,updatet,clicknum,todayclicknum,score,scorenum,introduction,yun1,m3u8,download,author_id) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
ok
INSERT INTO video (platform,name,img,jishu,othername,director,actor,genre,region,language,screen,mlen,updatet,clicknum,todayclicknum,score,scorenum,introduction,yun1,m3u8,download,author_id) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
ok
INSERT INTO video (platform,name,img,jishu,ot

In [7]:
con.commit()
con.close()

In [2]:
import time
time.time()-24*2*60*60


1586255787.749435

In [14]:
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(filename='./my.log', level=logging.DEBUG, format=LOG_FORMAT)
r=session.get('http://www.zuidazy5.com/')
for tempa in r.html.xpath("//a"):
    if tempa.text=="下一页":
        logging.error(tempa.attrs['href'])

ERROR:root:/?m=vod-index-pg-2.html


In [1]:
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(filename='./my.log', level=logging.DEBUG, format=LOG_FORMAT)

logging.debug("This is a debug log.")
logging.info("This is a info log.")
logging.warning("This is a warning log.")
logging.error("This is a error log.")
logging.critical("This is a critical log.")

In [8]:
searchsql='SELECT * from video where id=%d'%10
searchres=con.execute(searchsql).fetchall()

In [11]:
searchres[0]

(10,
 1,
 '风尘三侠之红拂女',
 'https://rpg.pic-imges.com/pic/upload/vod/2020-04/1586408259.jpg',
 '完结',
 '红拂女',
 '崔宝珠,侯继元,任海滔,梁德华',
 '舒淇,霍建华,江华,杨渝渝,姚采颖,郑则仕,李乐,于荣光,贾乃亮,侯勇,庞庸之,罗湘晋,刘芸,司光敏,董志华,张春仲,秦焰,铁鹤,王星凯,白玉,刘新渝,董艺薇,张宇,张熙,蔡小龙,赵冬柏,郭明翰,侯桐江,李方之,郭鑫,舒丽,吴东明,于文',
 '国产剧 古装',
 '大陆',
 '国语',
 '2006',
 '45',
 '2020-04-09 12:57:39',
 '0',
 '0',
 '0',
 '0',
 '红拂（舒淇饰）是隋朝大司空杨素的家伎，她不但貌美聪慧，更有一身过人武艺。杨素为巩固势力，在自己的司空府地下，建造了一座神秘地下城。杀手之王独孤城（江华饰）是他的义子，也是负责管理这座地下城的人。红拂父母亲被杀后，独孤城将幼小的红拂带回地下城，并且都她武艺，使她成为地宫暗人。红拂逐渐长大，她所有的少女情怀都寄托在独孤城身上。然而夺去她童贞的却是杨素。杨素把持朝纲，一手遮天，为了铲除异己，他训练暗人大肆杀戮。并命令“阴世师”用活人残忍的练制刀枪不入，力大无穷的杀人机器--战奴。一时间，朝廷内外血雨腥风，动荡不堪…… 大司空府的招贤大会上，满腹经纶的李靖（霍建华饰）出尽风头。他不但破了杨素亲自设计的阵法，还赢得了红拂的青睐，杨素决定要杀李靖，并故意指派红拂去执行这个任务。红拂夜访李靖，劝他逃亡。杨素又派了抱琴与绿羽追杀李靖。红拂不得已现身护卫李靖，正式走上了背离地下宫的不归路。红拂与李靖一路逃避独孤城和抱琴的追杀，多次面对生死关头，地一人同心同命。逐渐培养了情感，李靖决定娶红拂为妻。两人在灵石镇简陋的客栈里举行婚礼，这时，顶尖高手虬髯客（于荣光饰）出现了，虬髯客与红拂，李靖一见如故，虬 髯客发现红拂与自己都姓张，于是两个结拜为兄妹，虬髯客家中排行三，红拂唤他“三哥”，而红拂行一，虬髯客唤她“一妹”，兄妹相知相惜，江湖人称“风尘三侠”。为逃避追杀，红拂一行人往瓦岗寨行去，被独孤城、抱琴追上，虬髯客与独孤城两个绝世高手展开了一场惊天动地的生死决杀，在混战中，抱琴使地下宫独门暗器炸毁了山崖，李靖和红拂双双落入山涧

In [19]:
a={'1':'a','2':'b','3':'c'}
for i in a.values():
    print(i)

a
b
c


In [36]:
a=c.execute('select count(*) from video').fetchall()[0][0]

In [37]:
a

307